In [1]:
import pandas as pd
import numpy as np
import duckdb
import warnings
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import (
    roc_auc_score, precision_recall_curve, average_precision_score,
    classification_report, confusion_matrix, precision_score, recall_score, f1_score
)
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import joblib
import warnings

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

# Feature Engineering

In [2]:
con = duckdb.connect(database=":memory:")
df = pd.read_csv("../tables/players_gamelogs/players_gamelogs.csv")
df2 = pd.read_csv("../tables/players_xref.csv")
df = con.execute("""SELECT df.*, df2.* EXCLUDE(Team, ABV, Position, Season, Player, pfr_id, gm_log_rtrvd) FROM df JOIN df2 ON df.pfr_id = df2.pfr_id""").fetchdf()
og_cols = df.columns.tolist()

df['injured_out'] = (df['Season_type'] == 'INJ').astype(int)
INJ_Gms = (
    df.groupby(['Player', 'Season'], as_index=False)['injured_out']
      .sum()
      .rename(columns={'injured_out': 'INJ_Gms'})
)
df = df.merge(INJ_Gms, on=['Player', 'Season'], how='left')
df['long_term_injury'] = (df['INJ_Gms'] >= 4).astype(int)
og_cols.append('INJ_Gms')
og_cols.append('long_term_injury')

df['gms_plyd'] = (df['Season_type'] == 'REG').astype(int)
gms_plyd = (
    df.groupby(['Player', 'Season'], as_index=False)['gms_plyd']
      .sum()
      .rename(columns={'gms_plyd': 'Gms'})
)
df = df.merge(gms_plyd, on=['Player', 'Season'], how='left')
og_cols.append('Gms')

df = df[og_cols]
display(df)

,CarGm,Team,Player,pfr_id,Date,Season,Week,Season_type,GS,RushAtt,RushYds,RushTD,Tgt,Rec,RecYds,RecTD,OffSnp,STSnp,SUSP,INJ,Game Status,Injuries,Birth_date,Age,Years_exp,Entry_year,Height,Weight,INJ_Gms,long_term_injury,Gms
0,0,GNB,Aaron Jones,JoneAa00,2017-09-10,2017,1,DNP,0,0,0,0,0,0,0,0,0,0,0,0,None,None,12/2/1994,29,7,2017,69,208,3,0,12
1,1,GNB,Aaron Jones,JoneAa00,2017-09-17,2017,2,REG,0,0,0,0,0,0,0,0,0,8,0,0,None,None,12/2/1994,29,7,2017,69,208,3,0,12
2,2,GNB,Aaron Jones,JoneAa00,2017-09-24,2017,3,REG,0,0,0,0,0,0,0,0,0,12,0,0,None,None,12/2/1994,29,7,2017,69,208,3,0,12
3,3,GNB,Aaron Jones,JoneAa00,2017-09-28,2017,4,REG,0,13,49,1,0,0,0,0,30,8,0,0,None,None,12/2/1994,29,7,2017,69,208,3,0,12
4,4,GNB,Aaron Jones,JoneAa00,2017-10-08,2017,5,REG,1,19,125,1,1,1,9,0,53,0,0,0,None,None,12/2/1994,29,7,2017,69,208,3,0,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58532,0,CIN,Zack Moss,MossZa00,2024-12-09,2024,14,INJ,0,0,0,0,0,0,0,0,0,0,0,1,Out,Inferred,12/15/1997,26,4,2020,70,215,10,1,8
58533,0,CIN,Zack Moss,MossZa00,2024-12-15,2024,15,INJ,0,0,0,0,0,0,0,0,0,0,0,1,Out,Inferred,12/15/1997,26,4,2020,70,215,10,1,8
58534,0,CIN,Zack Moss,MossZa00,2024-12-22,2024,16,INJ,0,0,0,0,0,0,0,0,0,0,0,1,Out,Inferred,12/15/1997,26,4,2020,70,215,10,1,8
58535,0,CIN,Zack Moss,MossZa00,2024-12-28,2024,17,INJ,0,0,0,0,0,0,0,0,0,0,0,1,Out,Inferred,12/15/1997,26,4,2020,70,215,10,1,8


In [3]:
df = df.groupby(['pfr_id', 'Player', 'Season', 'Gms', 'INJ_Gms', 'long_term_injury']).agg({
    'GS': 'sum',
    'OffSnp': 'sum',
    'STSnp': 'sum',
    'RushYds': 'sum',
    'RushAtt': 'sum',
    'RecYds': 'sum',
    'Tgt': 'sum'
}).reset_index()

df = con.execute("""SELECT df.*,     
                    COALESCE(CAST(SUM(Gms) OVER (PARTITION BY df.pfr_id ORDER BY df.Season
                    ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING) AS INT), 0) AS Gms_Last3, 
                    COALESCE(CAST(SUM(long_term_injury) OVER (PARTITION BY df.pfr_id ORDER BY df.Season
                    ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING) AS INT), 0) AS IRs_Last3, 
                    COALESCE(CAST(SUM(INJ_Gms) OVER (PARTITION BY df.pfr_id ORDER BY df.Season
                    ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING) AS INT), 0) AS GmsINJ_Last3, 
                    COALESCE(CAST(SUM(RushYds) OVER (PARTITION BY df.pfr_id ORDER BY df.Season
                    ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING) AS INT), 0) AS RushYds_Last3, 
                    COALESCE(CAST(SUM(RecYds) OVER (PARTITION BY df.pfr_id ORDER BY df.Season
                    ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING) AS INT), 0) AS RecYds_Last3, 
                    COALESCE(CAST(SUM(OffSnp) OVER (PARTITION BY df.pfr_id ORDER BY df.Season
                    ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) AS INT), 0) AS CareerOffSnp, 
                    COALESCE(CAST(SUM(STSnp) OVER (PARTITION BY df.pfr_id ORDER BY df.Season
                    ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) AS INT), 0) AS CareerSTSnp, 
                    COALESCE(CAST(SUM(long_term_injury) OVER (PARTITION BY df.pfr_id ORDER BY df.Season
                    ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) AS INT), 0) AS CareerIRs, 
                    COALESCE(CAST(SUM(Gms) OVER (PARTITION BY df.pfr_id ORDER BY df.Season
                    ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) AS INT), 0) AS CareerGms, 
                    COALESCE(CAST(SUM(RushAtt) OVER (PARTITION BY df.pfr_id ORDER BY df.Season
                    ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) AS INT), 0) AS CareerRushAtts, 
                    COALESCE(CAST(SUM(Tgt) OVER (PARTITION BY df.pfr_id ORDER BY df.Season
                    ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) AS INT), 0) AS CareerTgts, 
                    COALESCE(CAST(SUM(RushYds) OVER (PARTITION BY df.pfr_id ORDER BY df.Season
                    ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) AS INT), 0) AS CareerRushYds, 
                    COALESCE(CAST(SUM(RecYds) OVER (PARTITION BY df.pfr_id ORDER BY df.Season
                    ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) AS INT), 0) AS CareerRecYds, 
                    LAG(long_term_injury, 1, 0) OVER (PARTITION BY df.pfr_id ORDER BY df.Season) AS inj_season_before,
                    LEAD(long_term_injury) OVER (PARTITION BY df.pfr_id ORDER BY df.Season) AS pred_inj_nxt_szn,
                    Height, Weight, Birth_date, Entry_year FROM df JOIN df2 ON df.pfr_id = df2.pfr_id""").fetchdf()
df['Years_exp'] = df.Season - df.Entry_year
df['Birth_date'] = pd.to_datetime(df['Birth_date'])
df["season_dt"] = pd.to_datetime(df["Season"], format="%Y")
df['Age'] = ((df['season_dt'] - df['Birth_date']) / pd.Timedelta(days=365)).astype(int)
df = df.drop(['season_dt', 'Birth_date', 'Entry_year'], axis=1)

df = con.execute("""
    SELECT 
        df.*,
        -- Previous season stats per player
        LAG(RushAtt, 1, 0) OVER (PARTITION BY pfr_id ORDER BY Season) AS RushAtt_prev,
        LAG(Tgt, 1, 0) OVER (PARTITION BY pfr_id ORDER BY Season) AS Tgt_prev,

        -- Workload ratio (safe from divide-by-zero)
        CASE 
            WHEN (LAG(RushAtt, 1, 0) OVER (PARTITION BY pfr_id ORDER BY Season)
                 + LAG(Tgt, 1, 0) OVER (PARTITION BY pfr_id ORDER BY Season)) = 0 
            THEN 1.0
            ELSE 
                CAST((RushAtt + Tgt) AS DOUBLE) /
                NULLIF(
                    (LAG(RushAtt, 1, 0) OVER (PARTITION BY pfr_id ORDER BY Season)
                   + LAG(Tgt, 1, 0) OVER (PARTITION BY pfr_id ORDER BY Season)), 
                0)
        END AS Workload_ratio
    FROM df
""").fetchdf()
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df['Workload_ratio'].fillna(1, inplace=True)
df = df.drop(['RushAtt_prev', 'Tgt_prev'], axis=1)

df['Touches_per_gm'] = df.Tgt + df.RushAtt / df.Gms
df['OffSnp_missing'] = (df['Season'] < 2014).astype(int)
df['STSnp_missing'] = (df['Season'] < 2014).astype(int)
df.loc[df['Season'] < 2014, ['OffSnp','STSnp']] = np.nan


# df = df[['pfr_id', 'Player', 'Season', 'Height', 'Weight', 'Age', 'Years_exp', 'Season', 'Gms', 'GS', 'INJ_Gms', 
#          'long_term_injury', 'inj_season_before',
#          'RushYds', 'RushAtt', 'RecYds', 'Tgt', 'OffSnp', 'STSnp', 'OffSnp_missing', 'STSnp_missing',
#           'CareerIRs', 'CareerGms', 'CareerRushAtts', 'CareerTgts', 'CareerRushYds', 'CareerRecYds', 'CareerOffSnp', 'CareerSTSnp', 
#           'RecYds_Last3', 'RushYds_Last3', 'Gms_Last3', 'GmsINJ_Last3', 'IRs_Last3',
#          'Workload_ratio', 'Touches_per_gm', 'pred_inj_nxt_szn']]

df = df[['pfr_id', 'Player', 'Season', 'Gms', 'Years_exp', 'inj_season_before', 'Height', 'Weight', 'Age',
         'RushYds', 'RushAtt', 'RecYds', 'Tgt', 'RushYds_Last3', 'Gms_Last3', 'GmsINJ_Last3',
          'CareerIRs', 'CareerRushAtts', 'CareerTgts', 'CareerRushYds', 'CareerRecYds',  'pred_inj_nxt_szn']].sort_values(by=['pfr_id', 'Season']).reset_index(drop=True)

display(df)

,pfr_id,Player,Season,Gms,Years_exp,inj_season_before,Height,Weight,Age,RushYds,RushAtt,RecYds,Tgt,RushYds_Last3,Gms_Last3,GmsINJ_Last3,CareerIRs,CareerRushAtts,CareerTgts,CareerRushYds,CareerRecYds,pred_inj_nxt_szn
0,AbduAm00,Ameer Abdullah,2015,16,0,0,69,203,21,597,143,183,38,0,0,0,0,0,0,0,0,1
1,AbduAm00,Ameer Abdullah,2016,2,1,0,69,203,22,101,18,57,5,597,16,0,0,143,38,597,183,0
2,AbduAm00,Ameer Abdullah,2017,14,2,1,69,203,23,552,165,162,35,698,18,15,1,161,43,698,240,1
3,AbduAm00,Ameer Abdullah,2018,10,3,0,69,203,24,1,1,28,4,1250,32,17,1,326,78,1250,402,0
4,AbduAm00,Ameer Abdullah,2019,16,4,1,69,203,25,124,24,95,22,654,26,21,2,327,82,1251,430,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3272,ZereAm00,Amos Zereoue,2001,14,2,0,68,205,24,515,113,217,22,62,20,6,1,24,2,62,17,0
3273,ZereAm00,Amos Zereoue,2002,16,3,0,68,205,25,884,220,351,56,577,34,6,1,137,24,577,234,0
3274,ZereAm00,Amos Zereoue,2003,16,4,0,68,205,26,433,132,310,52,1413,42,0,1,357,80,1461,585,0
3275,ZereAm00,Amos Zereoue,2004,15,5,0,68,205,27,425,112,284,48,1832,46,0,1,489,132,1894,895,1


# Baseline Model

In [4]:
train_df = df[(df['Season'] <= 2021) & (df.pred_inj_nxt_szn != '<NA>')].copy()
val_df   = df[(df['Season'] == 2022) & (df.pred_inj_nxt_szn != '<NA>')].copy()
test_df  = df[(df['Season'] == 2023) & (df.pred_inj_nxt_szn != '<NA>')].copy()
pred_df  = df[(df['Season'] == 2024)].copy()
pred_df['pred_inj_nxt_szn'] = np.where(pred_df.pred_inj_nxt_szn.isnull(), np.nan, pred_df.pred_inj_nxt_szn)

feature_cols = [col for col in df.columns 
                if col not in ['pfr_id', 'Player', 'Season', 'pred_inj_nxt_szn']]

X_train, y_train = train_df[feature_cols], train_df['pred_inj_nxt_szn'].astype(int)
X_val,   y_val   = val_df[feature_cols],   val_df['pred_inj_nxt_szn'].astype(int)
X_test,  y_test  = test_df[feature_cols],  test_df['pred_inj_nxt_szn'].astype(int)

print(f"Class counts (train): 0 - {y_train.value_counts()[0]} / 1 - {y_train.value_counts()[1]}")
print(f"Class counts (val): 0 - {y_val.value_counts()[0]} / 1 - {y_val.value_counts()[1]}")
print(f"Class counts (test): 0 - {y_test.value_counts()[0]} / 1 - {y_test.value_counts()[1]}")

# Compute scale_pos_weight from training set 
neg = (y_train == 0).sum()
pos = (y_train == 1).sum()
scale_pos_weight = neg / pos
print(f"Computed scale_pos_weight = {scale_pos_weight:.3f} (neg/pos = {neg}/{pos})")

# Baseline XGBoost model with balanced-ish params
base_model = XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    use_label_encoder=False,
    max_depth=3,
    learning_rate=0.03,
    n_estimators=600,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=3,
    scale_pos_weight=scale_pos_weight,   # from train set
    reg_lambda=2,
    reg_alpha=0,
    random_state=42,
    n_jobs=-1
)

# Fit baseline (this gives you baseline AUC before hyperparam tuning)
base_model.fit(X_train, y_train)
val_proba = base_model.predict_proba(X_val)[:, 1]
print("Baseline Validation AUC:", roc_auc_score(y_val, val_proba))

Class counts (train): 0 - 1989 / 1 - 711
Class counts (val): 0 - 51 / 1 - 23
Class counts (test): 0 - 43 / 1 - 18
Computed scale_pos_weight = 2.797 (neg/pos = 1989/711)
Baseline Validation AUC: 0.6521739130434783


# Hyperparameter tuning

In [5]:
param_dist = {
    "max_depth": [2, 3, 4, 5],
    "learning_rate": [0.01, 0.03, 0.05],
    "n_estimators": [300, 600, 900],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "min_child_weight": [1, 3, 5, 8],
    "reg_lambda": [0.5, 1, 2, 5],
    # scale_pos_weight we leave anchored at computed value ± small multipliers to avoid overcorrection:
    "scale_pos_weight": [max(1.0, scale_pos_weight * 0.75), scale_pos_weight, scale_pos_weight * 1.25]
}

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
rs = RandomizedSearchCV(
    estimator=XGBClassifier(objective='binary:logistic', use_label_encoder=False, eval_metric='auc', n_jobs=-1, random_state=42),
    param_distributions=param_dist,
    n_iter=30,
    scoring='roc_auc',
    cv=cv,
    verbose=1,
    random_state=42
)

# Fit the search on training data
rs.fit(X_train, y_train)
print("Best params (AUC-optimized):", rs.best_params_)
best_model = rs.best_estimator_

# Evaluate best model on validation
val_proba = best_model.predict_proba(X_val)[:, 1]
val_auc = roc_auc_score(y_val, val_proba)
print("Validation AUC (best_model):", val_auc)
print("Validation AP (average precision):", average_precision_score(y_val, val_proba))

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best params (AUC-optimized): {'subsample': 0.8, 'scale_pos_weight': np.float64(3.4968354430379742), 'reg_lambda': 5, 'n_estimators': 600, 'min_child_weight': 8, 'max_depth': 2, 'learning_rate': 0.01, 'colsample_bytree': 0.8}
Validation AUC (best_model): 0.70076726342711
Validation AP (average precision): 0.6182135113347617


# Test set on final model

In [6]:
# Pick threshold automatically given a target recall or min precision constraint
target_recall = 0.65
possible = []
for t in np.linspace(0, 1, 201):
    yp = (val_proba >= t).astype(int)
    r = recall_score(y_val, yp)
    p = precision_score(y_val, yp, zero_division=0)
    possible.append((t, p, r))
possible_df = pd.DataFrame(possible, columns=['threshold', 'precision', 'recall'])
candidates = possible_df[possible_df.recall >= target_recall].sort_values(['precision', 'threshold'], ascending=[False, True])
if not candidates.empty:
    chosen = candidates.iloc[0]
    print("Auto-chosen threshold for target_recall=", target_recall, "->", chosen.to_dict())
else:
    print("No threshold reaches target recall on validation. You may need to lower the target or improve model.")

# Final evaluation on test set at chosen threshold
if not candidates.empty:
    final_t = float(chosen.threshold)
else:
    final_t = 0.5  # fallback
print('final_t:', final_t)

test_proba = best_model.predict_proba(X_test)[:, 1]
test_pred = (test_proba >= final_t).astype(int)
print("Test AUC:", roc_auc_score(y_test, test_proba))
print("Test classification report at threshold", final_t)
print(classification_report(y_test, test_pred))

# Confusion matrix, etc.
print("Test confusion matrix:\n", confusion_matrix(y_test, test_pred))

Auto-chosen threshold for target_recall= 0.65 -> {'threshold': 0.64, 'precision': 0.5769230769230769, 'recall': 0.6521739130434783}
final_t: 0.64
Test AUC: 0.7558139534883721
Test classification report at threshold 0.64
              precision    recall  f1-score   support

           0       0.85      0.79      0.82        43
           1       0.57      0.67      0.62        18

    accuracy                           0.75        61
   macro avg       0.71      0.73      0.72        61
weighted avg       0.77      0.75      0.76        61

Test confusion matrix:
 [[34  9]
 [ 6 12]]


# Predict the 2025 season from 2024 data

In [7]:
if not pred_df.empty:
    X_pred = pred_df[feature_cols]
    pred_df['inj_proba'] = best_model.predict_proba(X_pred)[:, 1]
    pred_df['inj_pred'] = (pred_df['inj_proba'] >= final_t).astype(int)
    print(f"If inj_proba is >= {final_t}, then inj_pred == 1")
    pred_df = pred_df[['Player', 'Season', 'inj_proba', 'inj_pred']].sort_values('inj_proba', ascending=False)
    display(pred_df.head(30))
    pred_df.to_csv("../tables/ml_predictions.csv", index=False)

If inj_proba is >= 0.64, then inj_pred == 1


,Player,Season,inj_proba,inj_pred
676,Dalvin Cook,2024,0.801216,1
1081,Myles Gaskin,2024,0.792665,1
872,Clyde EdwardsHelaire,2024,0.791653,1
1751,Joshua Kelley,2024,0.788562,1
586,Michael Carter,2024,0.774527,1
1958,Christian McCaffrey,2024,0.765470,1
2218,Raheem Mostert,2024,0.749560,1
3217,Jeff Wilson,2024,0.739541,1
1475,Kareem Hunt,2024,0.731984,1
624,Nick Chubb,2024,0.723513,1
